In [1]:
import numpy as np
import pandas as pd
storage_options = {'User-Agent': 'Mozilla/5.0'}
url = "https://www.fraserinstitute.org/sites/default/files/economic-freedom-of-north-america-2022-for-website-allgovs.xlsx"
GDP = pd.read_csv("StateGDP.csv")
efw = pd.read_excel(url, storage_options=storage_options, sheet_name=None)
codes = pd.read_csv("taxitemcodesanddescriptions.csv", header = [0], index_col = [0])

In [2]:
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

fips_dict = {
 '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR',
 '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE',
 '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID',
 '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS',
 '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
 '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
 '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV',
 '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY',
 '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK',
 '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC',
 '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
 '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
 '55': 'WI', '56': 'WY'}
# '60': 'AS', '66': 'GU',
#  '69': 'MP', '72': 'PR', '78': 'VI'}
state_df = pd.DataFrame(state_dict, index = ["Abbr"]).T
fips_df = pd.DataFrame(fips_dict, index = ["State"]).T
state_names, state_abbrev = state_df.index, list(state_df["Abbr"].values)
state_df.T# state_abbrev

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Abbr,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY


In [3]:
years = [1985,
 1990,
 1995,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]
efw = efw["Overall-scores(allgovs)"].dropna(subset = "Unnamed: 104").rename(columns = {"Overall allg Scores":"State"})
panel = efw[efw["Unnamed: 104"].str.contains(", US")]
panel = panel[["State"] + years]
panel["State"] = panel["State"].apply(lambda x:state_dict[x])
panel = panel.melt(id_vars=["State"], 
        var_name="Year", 
        value_name="EFW").set_index(["State", "Year"]).round(2)
panel

,,EFW
State,Year,
AL,1985,8.22
AK,1985,8.04
AZ,1985,8.25
AR,1985,8.20
CA,1985,8.07
...,...,...
VA,2020,7.94
WA,2020,7.90
WV,2020,7.87


In [4]:
GDP_years = list(GDP.keys())[-24:]
GDP = GDP[GDP["GeoName"].isin(state_names)]
GDP["State"] = GDP["GeoName"].apply(lambda x: state_dict[x])
GDP = GDP[["State","LineCode"] + GDP_years]
GDP = GDP.melt(id_vars=["State", "LineCode"], 
        var_name="Year", 
        value_name="GDP")

GDP["Year"] = GDP["Year"].astype(int)
GDP.set_index(["State", "Year"], inplace = True)
panel.loc[panel.index.isin(GDP.index),"GDP"] = GDP["GDP"][GDP["LineCode"]==1].astype(float)
panel

,,EFW,GDP
State,Year,,
AL,1985,8.22,NaN
AK,1985,8.04,NaN
AZ,1985,8.25,NaN
AR,1985,8.20,NaN
CA,1985,8.07,NaN
...,...,...,...
VA,2020,7.94,551760.3
WA,2020,7.90,618704.9
WV,2020,7.87,73709.2


In [5]:
# Maybe labels from 97 keys were entered by an intern who was filling in for the usual data person who was on leave,
fix_97_keys = pd.DataFrame({'VI':"VA", "AR":"AZ", 
                            'AR    .1':"AR", "LS":"LA",
                            "GE":"GA"}, 
                           index = ["Correction"]).T
fix_97_keys

,Correction
VI,VA
AR,AZ
AR .1,AR
LS,LA
GE,GA


In [6]:
# urls = ["https://www2.census.gov/govs/statetax/" +str(i)[-2:] + "staxcd.txt" for i in range(92,122)]
tax_dfs = {}
for year in range(1992,2022):
    key = str(year)[-2:] 
    try:    
        url = "https://www2.census.gov/govs/statetax/" +key + "staxcd.txt"
        tax_dfs[year] = pd.read_csv(url, index_col = [0])
        tax_dfs[year].rename(columns = {col:col.strip().replace("-State","") for col in tax_dfs[year].columns}, inplace = True)
        tax_dfs[year].rename(columns = {name:state_df.loc[name].values[0] for name in state_names}, inplace = True)
        if key == "97":
            tax_dfs[year].rename(columns = {
                wrong_key:fix_97_keys.loc[wrong_key].values[0] for wrong_key in fix_97_keys.index}, 
                                inplace = True)
        # adjust units from 1000s to millions
        tax_dfs[year] = tax_dfs[year][state_abbrev].div(1000)
#         tax_dfs[key].rename(columns = {col:col.strip() for col in tax_dfs[key].columns}, inplace = True)
    except:
        print("failed to download " + url)
tax_dfs.keys()

failed to download https://www2.census.gov/govs/statetax/16staxcd.txt
failed to download https://www2.census.gov/govs/statetax/17staxcd.txt
failed to download https://www2.census.gov/govs/statetax/18staxcd.txt
failed to download https://www2.census.gov/govs/statetax/19staxcd.txt
failed to download https://www2.census.gov/govs/statetax/20staxcd.txt
failed to download https://www2.census.gov/govs/statetax/21staxcd.txt


dict_keys([1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [7]:
code_dfs = {}
for code in codes.index:
    code_dfs[code] = {}
    for year in tax_dfs.keys():
        code_dfs[code][year] = tax_dfs[year].loc[code] 
    code_dfs[code] = pd.DataFrame(code_dfs[code])

In [8]:
code_dfs["TotalTax"] = {}

for key, df in tax_dfs.items():
    code_dfs["TotalTax"][key] = df.sum()
code_dfs["TotalTax"]

{1992: AL     1602.937
 AK     4826.755
 AZ     2748.292
 AR    47972.949
 CA     3531.698
 CO     6059.339
 CT     1341.005
 DE    14504.207
 FL     7266.981
 GA     2709.518
 HI     1396.013
 ID    13463.435
 IL     6963.139
 IN     3616.444
 IA     2801.692
 KS     5080.971
 KY     4250.245
 LA     1670.488
 ME     6502.494
 MD     9903.246
 MA    11279.170
 MI     7449.787
 MN     2494.392
 MS     5131.360
 MO     1034.876
 MT     1889.877
 NE     1823.376
 NV      985.415
 NH    12802.662
 NJ     2237.656
 NM    30113.133
 NY     9009.742
 NC      693.007
 ND    12114.788
 OH     3765.272
 OK     3322.287
 OR    16269.988
 PA     1307.111
 RI     3935.500
 SC      565.032
 SD     4858.098
 TN    17024.494
 TX     1987.793
 UT      765.972
 VT     7025.345
 VA     8476.932
 WA     2351.858
 WV     7389.207
 WI      645.929
 WY     2439.667
 dtype: float64,
 1993: AL     4712.127
 AK     2240.927
 AZ     5352.567
 AR     2982.265
 CA    50115.756
 CO     3863.303
 CT     6767.182
 D

In [9]:
tax_names = list(code_dfs.keys())
for code, df in code_dfs.items():
    print(code)
    print(df)
    code_dfs[code] = pd.DataFrame(df)
    code_dfs[code].index.name = "State"
    code_dfs[code] = code_dfs[code].reset_index().melt(id_vars=["State"], 
        var_name="Year", 
        value_name=code).set_index(["State", "Year"])#     panel.loc[panel.index.isin(code_dfs[code].index),code] = code_dfs[code]
    panel[code + "/GDP"] = panel[code].div(panel["GDP"]).mul(100)

T01
        1992      1993      1994      1995      1996      1997      1998  \
AL    69.048   101.135   112.596   118.202   134.252   131.229   139.818   
AK   288.356    66.918    61.612    57.294    56.168    53.567    48.516   
AZ    11.089   327.279   313.841   331.090   363.315   256.879   252.153   
AR  2946.074     4.868     5.708     6.395     7.363     8.355     9.091   
CA     7.382  2931.667  2999.953  3176.874  3375.264  3611.595  3870.605   
CO     0.012     8.618     8.597     6.614     0.000     0.000     8.138   
CT     0.000     0.012     0.015     0.012     0.038     0.012     0.013   
DE   490.073     0.000     0.000     0.000     0.000     0.000     0.000   
FL    27.758   666.360   681.484   706.021   754.482   787.614   984.694   
GA     0.000    31.726    33.779    32.893    35.651    38.155    37.824   
HI     0.012     0.000     0.000     0.000     0.000     0.000     0.000   
ID   223.973     0.109     0.000     0.111     0.000     0.000     0.000   
IL     4

KeyError: 'T01'

In [ ]:
# import matplotlib.pyplot as plt
# plot_df = panel.groupby("Year").sum().loc["2000":"2015"]
# tax_names = list(code_dfs.keys())
# fig, ax = plt.subplots(figsize = (20,10))
# plot_df[tax_names[:-1]].plot.area(
#     legend = False, 
#     ax = ax)
# plot_df[tax_names[-1]].plot.line(ls = "--", 
#                                  linewidth = 5, 
#                                  color = "k",
#                                  ax = ax)
plot_df

In [ ]:
tax_gdp_names = [name + "/GDP" for name in tax_names]
fig, ax = plt.subplots(figsize = (20,10))
plot_df[tax_gdp_names[:-1]].plot.area(
    legend = False, 
    ax = ax)
plot_df[tax_gdp_names[-1]].plot.line(ls = "--", 
                                 linewidth = 5, 
                                 color = "k",
                                 ax = ax)

In [ ]:
plot_df

In [ ]:

fig, ax = plt.subplots(figsize = (20,20))
pd.plotting.scatter_matrix(panel[["EFW","GDP","TotalTax/GDP","T01/GDP"]].diff(5)
                           , alpha = .5, s = 200 / 3, ax=ax)  
panel[["EFW","GDP","TotalTax/GDP","T01/GDP"]].corr()

In [ ]:
log_data = np.log(panel[["GDP","TotalTax","T01"]])
log_data.describe()